In [17]:
import requests

# Wikipedia日本語版のAPIエンドポイント
WIKI_API_URL = "https://ja.wikipedia.org/w/api.php"

# 検索キーワードに該当するWikipediaページを取得する関数
def fetch_search_results(keyword, limit=10):
    headers = {
        'User-Agent': 'MyWikipediaApp/1.0 (https://example.com; myemail@example.com)'
    }
    params = {
        "action": "query",
        "list": "search",
        "srsearch": keyword,
        "format": "json",
        "srlimit": limit
    }

    try:
        response = requests.get(WIKI_API_URL, params=params, headers=headers)
        response.raise_for_status()
        data = response.json()
        return data.get("query", {}).get("search", [])
    except requests.RequestException as e:
        print("検索リクエストでエラーが発生しました:", e)
        return []

# 指定されたタイトルのページ内容を取得する関数
def fetch_page_content(title):
    params = {
        "action": "query",
        "format": "json",
        "titles": title,
        "prop": "extracts",
        "explaintext": True
    }

    try:
        response = requests.get(WIKI_API_URL, params=params)
        response.raise_for_status()
        data = response.json()
        page = next(iter(data.get("query", {}).get("pages", {}).values()), {})
        return page.get("extract", "ページ内容が見つかりません。")
    except requests.RequestException as e:
        print("ページ内容取得でエラーが発生しました:", e)
        return "ページ内容が取得できませんでした。"

In [18]:
# テストコードで確認
keyword = "Python"

# キーワードに該当するページ一覧を取得
search_results = fetch_search_results(keyword)
#FIXME: 検索リクエストでエラーが発生しました: 403 Client Error: Forbidden for url: https://ja.wikipedia.org/w/api.php?action=query&list=search&srsearch=Python&format=json&srlimit=10
if search_results:
    print("【キーワードに該当するページ一覧】")
    for result in search_results[:3]:
        print(f"- {result['title']}")

    # 最初のページタイトルを取得
    first_title = search_results[0]["title"]
    print("\n【最初のページタイトル】")
    print(first_title)

    # 最初のページの内容を取得
    page_content = fetch_page_content(first_title)
    print("\n【ページ内容】")
    print(page_content[:500])
else:
    print("検索結果が見つかりません。")

【キーワードに該当するページ一覧】
- Python
- モンティ・パイソン
- IronPython

【最初のページタイトル】
Python
ページ内容取得でエラーが発生しました: 403 Client Error: Forbidden for url: https://ja.wikipedia.org/w/api.php?action=query&format=json&titles=Python&prop=extracts&explaintext=True

【ページ内容】
ページ内容が取得できませんでした。


In [19]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from openai import OpenAI

# 環境変数の取得
load_dotenv("../.env")

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [20]:
# 要約を行うプロンプトを作成
prompt = f"""
以下の文章を要約してください。

# 条件：
- 小学生にもわかるように
- 300文字程度

# 文章：
{page_content[:1000]}
"""

# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "user", "content": prompt},
    ],
    max_tokens=500,
    temperature=0.3
)

In [21]:
# LLMからの回答を表示
print(response.choices[0].message.content.strip())

この文章は、あるページの内容を読み取ることができなかったということを伝えています。つまり、何かの情報を探そうとしたけれど、その情報が見つからなかったり、読み込めなかったりしたということです。たとえば、インターネットでウェブサイトを開こうとしたけれど、エラーが出て見れなかったり、リンクが切れていたりすることがあります。このようなことは、時々起こることなので、また後で試してみるといいでしょう。何か問題があったときは、他の方法で情報を探したり、別のページを見てみるのも良いアイデアです。
